In [1]:
import pandas as pd
from ml import Model
data_youla=pd.read_csv('.datasets/youla_dataset.csv')
data_hh=pd.read_csv('.datasets/hh_dataset.csv')
model = Model()

In [2]:
%%time
text="белая футболка"
ids=model.knn_youla(text, 5)
print(ids)

Wall time: 222 ms


In [3]:
data_youla.iloc[ids][['title','descrirption']]

,title,descrirption
11638,Футболка и шорты,Белая футболка и черные шорты
20261,Рубашка белая,NaN
31946,Футболки 4 шт,Футболки по 150р 48-50
2527,Футболки,Бу за все 500 по отдельности200₽
2563,Футболка,Бу


In [4]:
%%time
text="системный администратор"
ids=model.knn_hh(text, 5)
print(ids)

Wall time: 24.9 ms


In [5]:
data_hh.iloc[ids][['id','name','description']]

,id,name,description
600,31349839,Системный администратор,Обязанности: Администрирование серверов под у...
17412,31269741,Системный администратор / специалист техническ...,Требования к кандидату: Ответственность в раб...
615,31067255,Системный администратор,Обязанности: Администрирование серверов Linux...
2,31142090,Руководитель/заместитель руководителя Управлен...,Обязанности: Администрирование ЛВС Настройка/...
751,31041109,Системный администратор,"Требования: Поддержка пользователей (1, 2 лин..."


In [6]:
text="белая футболка"
print(model.knn_dummy(text))
text="системный администратор"
print(model.knn_dummy(text))

youla
hh
